# Plotting and colors

We've already covered a lot on plotting in the previous tutorials. This tutorial is mostly a review of the material covered already, plus a few new tools to use.

<div class="alert alert-info">
    
Click [here](https://mybinder.org/v2/gh/sciris/sciris/HEAD?labpath=docs%2Ftutorials%2Ftut_plotting.ipynb) to open an interactive version of this notebook.
    
</div>


## Plots

### Basic plotting

First, let's make a plot using standard Matplotlib, then import Sciris and make the same plot:

In [ ]:
import numpy as np
import pylab as pl

n = 1000
x = np.arange(n)
y = np.random.rand(n)
c = np.sqrt(x*y)
kwargs = dict(x=x, y=y, s=100, c=c, alpha=0.2)

# Vanilla Matplotlib
f1 = pl.scatter(**kwargs)
pl.show()

# Chocolate Sciris
import sciris as sc
f2 = pl.scatter(**kwargs)
pl.show()

See how the Sciris version is much sharper? If Sciris detects that Jupyter is running (`sc.isjupyter()`), it automatically switches to the higher-resolution backend `'retina'`. But, if you really like blurry plots, you can manually set `sc.options(jupyter=False)`.

### Plotting styles

By default, Sciris uses higher resolution both for display and saving. But it also has two other built-in styles, `simple` and `fancy`, that you can call just like any other Matplotlib style:

In [ ]:
def demo_plot(label, npts=100, nlines=5):
    fig = pl.figure()
    for i in range(nlines):
        np.random.seed(i+1)
        pl.plot(np.cumsum(np.random.randn(npts)), alpha=0.7, label=f'Seed {i}')
    pl.title(f'Style {label}')
    pl.legend()
    pl.show()
    return

for style in ['default', 'sciris.simple', 'sciris.fancy']:
    with pl.style.context(style): # Use a style context so the changes don't persist
        demo_plot(style)

The "simple" style is close to Matplotlib's defaults (just without boxes around the axes and legend, more or less, while the "fancy" style is close to Seaborn's default.

### Plot configuration

One of the fun-but-annoying things about plots is that they're so customizable: no two plots are ever exactly the same. (One [hopes](https://www.nytimes.com/interactive/2022/10/29/opinion/science-fraud-image-manipulation-photoshop.html).) Sciris has a lot of options for configuring plots. Here are some of the most commonly used ones, which are hopefully more or less self-explanatory:

In [ ]:
sc.options(font='Garamond') # Set custom font
x = sc.daterange('2022-06-01', '2022-12-31', as_date=True) # Create dates
y = sc.smooth(np.random.randn(len(x))**2)*1000 # Create smoothed random numbers
c = sc.vectocolor(np.log(y), cmap='turbo') # Set colors proportional to squared y values
pl.scatter(x, y, c=c) # Plot the data
sc.dateformatter() # Automatic x-axis date formatter
sc.commaticks() # Add commas to y-axis tick labels
sc.setylim() # Automatically set the y-axis limits, including starting at 0
sc.boxoff() # Remove the top and right axis lines

### Advanced plotting options



Do you ever have, say, 14 plots, and have to think about how to turn that into a grid of subplots? `sc.getrowscols()` will solve that problem for you. Speaking of subplots, by default Matplotlib has a lot of wasted space; `sc.figlayout()` will convert the figure to "tight" layout, which (usually) fixes this. Finally, since 3D plots are cool, let's do more of those. Putting it all together:

In [ ]:
class Squiggles(sc.prettyobj):
    
    def __init__(self, n=9, length=100):
        self.n = n
        self.length = length

    def make(self):
        self.d = sc.objdict() # Create objdict to store the data
        for k in ['x', 'y', 'z']:
            self.d[k] = np.cumsum(np.random.randn(self.n, self.length), axis=1)
        self.c = sc.vectocolor(np.arange(self.length), cmap='parula') # Make colors
    
    def plot(self):
        d = self.d
        nrows,ncols = sc.getrowscols(self.n) # Automatically figure out the rows and columns
        pl.figure(figsize=(10,8))
        for i in range(self.n):
            ax = pl.subplot(nrows, ncols, i+1, projection='3d')
            sc.scatter3d(d.x[i], d.y[i], d.z[i], s=20, c=self.c, ax=ax, alpha=0.5) # Plot 3D
        sc.figlayout() # Automatically remove excess whitespace
        pl.show()
        return

sq = Squiggles()
sq.make()
sq.plot()

In [ ]:
sc.pr(sq)